# Aerodynamic Splatting

In [1]:
import numpy as np
from sympy import symbols, diff, Quaternion, exp, Matrix, lambdify, sin, cos, simplify
#from sympy.physics.mechanics import *
import random
from IPython.display import display
import matplotlib.pyplot as plt
from aerosplat import *

## Symbolic Math

## Equation for quaternion
We provide two options for obtaining the quaternon.
In the case where a `numpy` array with four quaternion components are provided, we will obtain using the standard syntax `Quaternion(qw, qx, qy, qz)`.
In the case where a single component is provided, we will assume that this is a planar model with an angle evaluated about the $z$-axis, and obtain with the `Quaternion.from_axis_angle` class method.
We also will create the `idx_dimension` variable which shrinks subsequent matrices and vectors to use only the first two ($x$ and $y$) components.

In [ ]:
display(quaternion_2d)

In [ ]:
display(quaternion_3d)

## Equation for gaussian

The Gaussian equation that is created below uses the variance term _not-squared_, while in the plot I square this term.
There is reason for this; the variance that I will use below is going to be of the form

$\texttt{variance} = \sum_{i=1}^3 \left( \frac{x_i - \bar{x}_i}{s_i} \right)^2$

where $x_i$ is a position coordinate for where we are measuring, $\bar{x}_i$ is the position coordinate for the center of the Gaussian, and $s_i$ is a scale coefficient representing one standard deviation in the direction of that coordinate.
Traditionally, we would evaluate the Gaussian at some count of standard deviations, which would derive from the square-root of the above expression.
To avoid taking a square-root, just to subsequently square the term again, we create the Gaussian as a function of the variance directly.
However, the plot below shows an evaluation of the Gaussian with respect to standard deviations.

In [ ]:
stdev = np.linspace(-3, 3, 121);
plt.plot(stdev, np.exp(-0.5 * stdev**2));
plt.xlabel("Standard Deviations");
plt.ylabel("Gaussian");

Here we create a function that obtains the rotation matrix from the quaternion, along with some simplification steps.
First, we substitute a value of 1 for the quaternion norm, as we will ensure that it is always a proper unit quaternion.
Second, we call the standard `simplify()` function.

In [ ]:
display(rotation_eqn_2d)

In [ ]:
display(rotation_eqn_3d)

Velocity away from the center of the Gaussian takes the form

$\boldsymbol{x} = (x, y, z, u, v, w, s_x, s_y, s_z, q_w, q_x, q_y, q_z)$

$\boldsymbol{x}_p = (x_p, y_p, z_p)$

$\boldsymbol{v(\boldsymbol{x}, \boldsymbol{x}_p)} = \boldsymbol{v_0(\boldsymbol{x})} g(\boldsymbol{x}, \boldsymbol{x}_p)$

In [ ]:
display(Matrix(velocity_eqn_2d))

We see the variance contained in the exponent

In [ ]:
display(variance_eqn_2d)

### Velocity derivative equations

Velocity is 1x2 or 1x3

Variance derivates are 1xN

$$
\frac{\partial \boldsymbol{v(\boldsymbol{x}, \boldsymbol{x}_p)} }{\partial x_i}
= 
\frac{\partial \boldsymbol{v_0(\boldsymbol{x})}}{\partial x_i}
g(\boldsymbol{x}, \boldsymbol{x}_p)
+
\boldsymbol{v_0(\boldsymbol{x})}
\frac{\partial g(\boldsymbol{x}, \boldsymbol{x}_p)}{\partial x_i}
$$

## Splatting Class

In [ ]:
a = AeroSplat(
    position=[0, 0], 
    velocity=[1, 0],
    scale=[1, 1],
    orientation=[0]
)
display(a)
display(a.quaternion)
display(a.rotation_matrix)
display(a.velocity_at(np.array([2, 0])))
display(a.differential_velocity_at(np.array([2, 0])))

## Boundary Condition Class

In [ ]:
source_boundary = LineBoundary(point0=[0, -1], point1=[0, 1], velocity=[10, 0])
plate_boundary = LineBoundary(point0=[1, -0.1], point1=[1, 0.1], velocity=[0, 0])
print(source_boundary)
print(source_boundary.point_at_random())

## Problem Definition Class

In [ ]:
problem = AeroSplatProblem(
    domain_x=[0, 2], 
    domain_y=[-1, 1],
    boundaries=[source_boundary, plate_boundary],
    spawn=20
)
print(problem)

In [ ]:
print(problem.velocity_on_grid(2, 2))
print(problem.ndims)
print(problem.splats)

In [ ]:
splat_positions = np.array([[splat.position[0], splat.position[1]] for splat in problem.splats])
plt.plot(splat_positions[:, 0], splat_positions[:, 1], "o");
plt.xlim(problem.domain_x);
plt.ylim(problem.domain_y);

In [ ]:
nx = 101
ny = 51
grid = problem.point_grid(nx, ny)
velocity_field = problem.velocity_on_grid(nx, ny)
contour_x = np.array([grid[0, k, 0] for k in range(nx)])
contour_y = np.array([grid[j, 0, 1] for j in range(ny)])
contour_z = np.array([[np.linalg.norm(velocity_field[j, k, :]) for k in range(nx)] for j in range(ny)])
plt.contourf(contour_x, contour_y, contour_z, cmap="turbo");
plt.colorbar();

In [ ]:
gradient_field = problem.velocity_gradient_on_grid(nx, ny)
contour_z = np.array([[np.linalg.norm(gradient_field[j, k, :]) for k in range(nx)] for j in range(ny)])
plt.contourf(contour_x, contour_y, contour_z, cmap="turbo");
plt.colorbar();